# Example 3: Urban climate adaption


In [1]:
from pyclmuapp import pts_clmu
from pyclmuapp import usp_clmu
from pyclmuapp.getcity import *
import matplotlib.pyplot as plt
import pandas as pd
import os

## 1 PTS mode: using the CESM provide global data


In [2]:
# the mask if from UrbanClimateExplorer/docs/notebooks/CESM1_get_urban_areas_mask.ipynb
# https://github.com/zhonghua-zheng/UrbanClimateExplorer/blob/main/docs/notebooks/CESM1_get_urban_areas_mask.ipynb
mask = pd.read_csv("src/CESM1_urban_mask.csv", index_col=0)
citylist = get_mask_cities(mask)


# set the location of London
London = {'lat': 51.5, 'lon': -0.1}

# find the closest city grid point to London (resolution is 1 degree, CESM nominal resolution)
closecity = closest(citylist,London)

closecity

{'lat': 51.361255645752, 'lon': 0.0}

In [3]:
# initialize the pts_clmu object

pts = pts_clmu(
    pwd=None,
    input_path="/home/zhonghua/p/scratch/CESMDATAROOT/inputdata",
    output_path="/home/zhonghua/p/scratch/CESMDATAROOT/Archive/lnd/hist",
    log_path="/home/zhonghua/p/scratch/CESMDATAROOT/CaseOutputs",
    scripts_path = "/home/zhonghua/p/scripts",
    container_type='docker')

# set the case configuration
pts.caseconfig['case_name'] = 'London'
pts.caseconfig['case_length'] = "3"
pts.caseconfig['hist_nhtfrq'] = '-24'
pts.caseconfig['STOP_OPTION'] = "nmonths"
pts.caseconfig['hist_mfilt']='240000'
pts.caseconfig['RUN_STARTDATE'] = '2012-06-01'
pts.caseconfig['DATM_CLMNCEP_YR_START'] = '2012'
pts.caseconfig['DATM_CLMNCEP_YR_END'] = '2012'
pts.caseconfig['case_lat'] = str(closecity['lat'])
pts.caseconfig['case_lon'] = str(closecity['lon'])
# run the case
pts_or=pts.run(iflog=True, cmdlogfile='./pts_London.log')

---

modify the whole urban roof albedo --> similate the white/cooling roof for urban adaptation

In [4]:
pts.modify_surf(var="ALB_ROOF_DIR", action=0.2) # Check the Python API documentation for more details on the parameters
pts_surf = pts.run(iflog=True, cmdlogfile='pts_London_surf.log')

The surface input file has been modified. The modified file is /p/scratch/CESMDATAROOT/inputdata/lnd/clm2/surfdata_map/release-clm5.0.18/surfdata_0.9x1.25_hist_16pfts_Irrig_CMIP6_simyr2000_c190214_modified_London.nc.


In [5]:
forcing = pts.input_path+"/atm/datm7/atm_forcing.datm7.cruncep_qianFill.0.5d.v7.c160715" + \
         "/TPHWL6Hrly" + "/clmforc.cruncep.V7.c2016.0.5d.TPQWL.2012-{month}.nc"
action = {"TBOT":1.0}
action_r = {"TBOT":-1.0}

def modify_forcing(forcing, action):
    for month in range(6, 9):
        forcing_file = forcing.format(month=str(month).zfill(2))
        pts.modify_forcing(var="None", action=action, forcing_location=forcing_file)

modify_forcing(forcing, action)

# get the result
pts_warming_surf = pts.run(iflog=True, cmdlogfile='./pts_London.log')

# recover the surface
pts.modify_surf(var="ALB_ROOF_DIR", action=-0.2)

# get the result
pts_warming = pts.run(iflog=True, cmdlogfile='./pts_London.log')

# recover the forcing
modify_forcing(forcing, action_r)

The surface input file has been modified. The modified file is /p/scratch/CESMDATAROOT/inputdata/lnd/clm2/surfdata_map/release-clm5.0.18/surfdata_0.9x1.25_hist_16pfts_Irrig_CMIP6_simyr2000_c190214_modified_London.nc.


Print dataset dataset location

In [6]:
print(pts_or)
print(pts_surf)
print(pts_warming)
print(pts_warming_surf)

['/home/zhonghua/p/scratch/CESMDATAROOT/Archive/lnd/hist/London_hist0_2024-06-17_16-04-18_clm.nc']
['/home/zhonghua/p/scratch/CESMDATAROOT/Archive/lnd/hist/London_hist0_2024-06-17_16-04-45_clm.nc']
['/home/zhonghua/p/scratch/CESMDATAROOT/Archive/lnd/hist/London_hist0_2024-06-17_16-05-44_clm.nc']
['/home/zhonghua/p/scratch/CESMDATAROOT/Archive/lnd/hist/London_hist0_2024-06-17_16-05-17_clm.nc']


In [7]:
pts.case_clean()